In [4]:
import bottleneck
import cdsapi
import dask
import geopandas as gpd
import netCDF4
import numexpr
import numpy as np
import pandas as pd
import tqdm
import rasterio
import rtree
import scipy
import shapely
import xarray as xr
import atlite
import logging

logging.basicConfig(level=logging.INFO)

In [9]:
import atlite
import pandas as pd
import geopandas as gpd
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np

# 1. Create a cutout for the entire Earth with ERA5 at hourly resolution from 01-01-2023 to 31-12-2023.
# Define the global bounds using unary_union of all countries
shpfilename = shpreader.natural_earth(
    resolution="10m", category="cultural", name="admin_0_countries"
)
reader = shpreader.Reader(shpfilename)
all_countries = gpd.GeoDataFrame(
    {'geometry': [r.geometry for r in reader.records()], 'country': [r.attributes['NAME_EN'] for r in reader.records()]},
    crs="EPSG:4326"
)

# Get the bounds of the entire world
global_bounds = all_countries.unary_union.bounds

# Define the time period of interest with hourly resolution
time_range = pd.date_range("2023-01-01", "2023-12-31 23:00:00", freq="h")

# Create the global cutout
cutout_global = atlite.Cutout(
    path="global-2023-era5-hourly.nc",  # Path where the cutout file will be saved
    module="era5",                      # The weather data source
    bounds=global_bounds,               # The geographical bounds from the union of all countries
    time=time_range                     # The time range with hourly resolution
)

# Prepare the cutout (this will download the necessary data and process it)
cutout_global.prepare()

INFO:atlite.cutout:Building new cutout global-2023-era5-hourly.nc
INFO:atlite.data:Storing temporary files in /var/folders/ds/cj7372k1487ccc2t6pwv92m80000gn/T/tmp9139ae6i
INFO:atlite.data:Calculating and writing with module era5:


Exception: Missing/incomplete configuration file: /Users/frederickivens/.cdsapirc